In [ ]:
import os
import pprint
import re
import time
import numpy as np
from numpy.lib import recfunctions as rfn
from pathlib import Path
from pxr import Usd, UsdGeom, Vt, Gf
from pybg3 import pak, lsf, _pybg3

In [ ]:
def checktime(msg, cb):
    t_start = time.time()
    rv = cb()
    t_end = time.time()
    print(f"{msg}: {t_end - t_start}")
    return rv

In [ ]:
BG3_ROOT = Path(os.environ.get("BG3_DATA", os.path.expanduser("~/l/bg3/Data")))
GUSTAV = checktime("Gustav.pak", lambda: pak.PakFile(BG3_ROOT / "Gustav.pak"))
SHARED = checktime("Shared.pak", lambda: pak.PakFile(BG3_ROOT / "Shared.pak"))
ENGINE = checktime("Engine.pak", lambda: pak.PakFile(BG3_ROOT / "Engine.pak"))
MODELS = checktime("Models.pak", lambda: pak.PakFile(BG3_ROOT / "Models.pak"))
VIRTUAL_TEXTURES = checktime("VirtualTextures.pak", lambda: pak.PakFile(BG3_ROOT / "VirtualTextures.pak"))

In [ ]:
data = GUSTAV.file_data("Generated/Public/GustavDev/Assets/HLOD/BGH_SteelWatchFoundry_B/HLOD_1_0_0_1.GR2")
granny = _pybg3._GrannyReader.from_data(data)

In [ ]:
root = granny.root
for name in dir(root):
  print(f"{name}: {root.__getattr__(name)}")
print(dir(root.ArtToolInfo))
print(root.ArtToolInfo.ArtToolPointerSize)
print(root.ArtToolInfo.UpVector)

In [ ]:
for mesh in root.Meshes:
  print(mesh.Name)
  vd = mesh.PrimaryVertexData
  nda = np.array(vd.Vertices, copy = False)
  new_arr = rfn.structured_to_unstructured(nda[["f0", "f1", "f2"]], copy = False)
  print(vd.VertexComponentNames)
  print(f"{vd.Vertices}: {len(vd.Vertices)} vertices")
  print(f"vertex data: {nda}")
  print(f"vertex data shape: {nda.shape}")
  print(f"vertex data dtype: {nda.dtype}")
  print(f"reshaped vertex data: {new_arr} {new_arr.shape} {new_arr.dtype}")
  print(dir(vd))

In [ ]:
g_mesh = root.Meshes[0]
g_vertices = np.array(g_mesh.PrimaryVertexData.Vertices, copy = False)
g_indices = np.array(g_mesh.PrimaryTopology.Indices16, copy = False)
print(g_mesh.Name)
pprint.pp(dir(g_mesh.PrimaryVertexData.Vertices[0]))

In [ ]:
u_stage = Usd.Stage.CreateNew("tmp/granny_test.usda")
u_mesh = UsdGeom.Mesh.Define(u_stage, "/mesh")

In [ ]:
u_points = u_mesh.CreatePointsAttr()
g_positions = rfn.structured_to_unstructured(g_vertices[["f0", "f1", "f2"]], copy = False)
vt_vertices = Vt.Vec3fArray.FromNumpy(g_positions)
vt_indices = Vt.IntArray.FromNumpy(g_indices)
vt_face_counts = Vt.IntArray.FromNumpy(np.full(g_positions.shape[0], 3, dtype = np.int32))
u_points.Set(vt_vertices)
u_vertex_face_counts = u_mesh.CreateFaceVertexCountsAttr()
u_vertex_face_counts.Set(vt_face_counts)
u_vertex_indices = u_mesh.CreateFaceVertexIndicesAttr()
u_vertex_indices.Set(vt_indices)

In [ ]:
u_stage.GetRootLayer().Save()

In [ ]:
for mesh in root.Meshes:
  print(mesh.Name)

In [ ]:
squirrel = MODELS.file_data("Generated/Public/Shared/Assets/Nature/Plants/NAT_Coastal_Plant_Bush_Pine_ABC/Resources/NAT_Coastal_Plant_Bush_Pine_Small_A.GR2")
print(len(squirrel))
g_squirrel = _pybg3._GrannyReader.from_data(squirrel)
print(dir(g_squirrel.root))
for mesh in g_squirrel.root.Meshes:
  print(mesh.Name)
for model in g_squirrel.root.Models:
  print(model.Name)

In [ ]:
Gf.Quatf(0,0,0,1)

In [ ]:
patch_data = GUSTAV.file_data("Mods/Gustav/Levels/TUT_Avernus_C/Terrains/2b55a30d-aca1-447f-8f4e-4d9215958607_2_0.patch")
patch = _pybg3._PatchFile.from_data(patch_data)

In [ ]:
height = np.array(patch.heightfield, copy = False)
print(height.shape)
print(np.array(patch.heightfield, copy = False))

In [ ]:
points = np.dstack((
  np.repeat(
    np.arange(0, height.shape[0]).reshape(-1, 1),
    height.shape[1],
    axis = 1
  ),
  height,
  np.repeat(
    np.arange(0, height.shape[1]).reshape(1, -1),
    height.shape[0],
    axis = 0
  ),
)).reshape(-1, 3)

In [ ]:
points

In [ ]:
# copilot lawl
indices = np.arange(patch.local_rows * patch.local_cols).reshape(patch.local_rows, patch.local_cols)
triangles = np.array([indices[:-1, :-1], indices[:-1, 1:], indices[1:, :-1], indices[1:, :-1], indices[:-1, 1:], indices[1:, 1:]])
index_buffer = triangles.transpose(1, 2, 0).reshape(-1)

print(index_buffer)

In [ ]:
GUSTAV._lspk.num_parts()

In [ ]:
def vt_test():
  for name in VIRTUAL_TEXTURES.files():
    if VIRTUAL_TEXTURES.file_part(name) > 0:
      print(name)
      data = VIRTUAL_TEXTURES.file_data(name)
      os.makedirs(f"tmp/{os.path.dirname(name)}", exist_ok = True)
      with open(f"tmp/{name}", "wb") as f:
        f.write(data)
      break
vt_test()

In [ ]:
#for name in VIRTUAL_TEXTURES.files():
#  if name.endswith(".gts"):
#    print(f"granite tileset: {name} {VIRTUAL_TEXTURES.file_size(name)}")
#    os.makedirs(f"tmp/{os.path.dirname(name)}", exist_ok = True)
#    with open(f"tmp/{name}", "wb") as f:
#      f.write(VIRTUAL_TEXTURES.file_data(name))

In [ ]:
gts = _pybg3._GtsReader.from_path("tmp/Generated/Public/VirtualTextures/Albedo_Normal_Physical_0.gts")

In [ ]:
gts.dump()